In [3]:
using PowerSystems
using InfrastructureSystems
using PowerSimulations
using HydroPowerSimulations
using HiGHS # solver
using Dates

[info | PowerModels]: extending matpower format with constant data: const_int
[info | PowerModels]: extending matpower format with constant data: const_float
[info | PowerModels]: extending matpower format with data: bus_data 3x3
[info | PowerModels]: extending matpower format with data: branch_names 3x3
[info | PowerModels]: extending matpower format with data: areas_cells 2x6
[info | PowerModels]: extending matpower format with data: areas_named_cells 2x6
[info | PowerModels]: extending matpower format with data: areas_named 2x3
[info | PowerModels]: extending matpower format with data: component 1x3
[info | PowerModels]: extending matpower format with data: areas 2x3
[info | PowerModels]: extending matpower format with constant data: const_str
[info | PowerModels]: extending matpower format with data: load_data 2x2
[info | PowerModels]: extending matpower format with data: branch_limit 3x3
[warn | PowerModels]: added zero cost function data for dclines
[info | PowerModels]: extendin

In [4]:
file_dir = joinpath(homedir(), "sienna_data", "final_sys_DA.json")
sys = System(file_dir)

┌ Info: Loaded time series from storage file existing=/home/emco4286/sienna_data/final_sys_DA_time_series_storage.h5 new=/projects/emco4286/code-server/tmp/jl_OSPfMm compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
└ @ InfrastructureSystems /home/emco4286/.julia/packages/InfrastructureSystems/LEg3t/src/hdf5_time_series_storage.jl:110
┌ Warning: Invalid range
│   valid_info.struct_name = ThermalMultiStart
│   field_name = active_power
│   valid_range = active_power_limits
│   valid_info.ist_struct = ThermalMultiStart(DEER_PARK_ENERGY_CENTER_CC4, true, true, ACBus(7113, DEER PARK 9, ACBusTypes.PV = 2, -0.3899502230334181, 1.04, (min = 0.9, max = 1.1), 24.0, Area(Coast, 202.78914140625, 0.0, 0.0, Dict{String, Any}()), LoadZone(4, 43.636300000000006, 12.363500000000004, Dict{String, Any}()), Dict{String, Any}("x" => -95.12381600000002, "y" => 29.710366140597316)), 0.006102362204724409, 0.4178, 9.4488, PrimeMovers.CC = 4, ThermalFuels.NATURAL_GAS = 7, (min = 6.2

Property,Value
Name,
Description,
System Units Base,SYSTEM_BASE
Base Power,100.0
Base Frequency,60.0
Num Components,10247
Type,Count
ACBus,2103
Arc,2834
Area,8


In [5]:
template_uc = template_unit_commitment(network=NetworkModel(CopperPlatePowerModel, use_slacks = true))

# set_device_model!(template_uc, Line, StaticBranch)
# set_device_model!(template_uc, Transformer2W, StaticBranch)
# set_device_model!(template_uc, TapTransformer, StaticBranch)

# set_device_model!(template_uc, ThermalStandard, ThermalStandardUnitCommitment)
# set_device_model!(template_uc, RenewableDispatch, RenewableFullDispatch)
# # set_device_model!(template_uc, EnergyStorage, RenewableFullDispatch)
# set_device_model!(template_uc, PowerLoad, StaticPowerLoad)
# set_device_model!(template_uc, HydroDispatch, HydroDispatchRunOfRiver)
# set_device_model!(template_uc, RenewableNonDispatch, FixedOutput)

# set_service_model!(template_uc, VariableReserve{ReserveUp}, RangeReserve)
# set_service_model!(template_uc, VariableReserve{ReserveDown}, RangeReserve)

# set_network_model!(template_uc, NetworkModel(CopperPlatePowerModel, use_slacks = true))
solver = optimizer_with_attributes(HiGHS.Optimizer, "mip_rel_gap" => 0.5)

MathOptInterface.OptimizerWithAttributes(HiGHS.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("mip_rel_gap") => 0.5])

In [6]:
problem = DecisionModel(template_uc, sys; optimizer = solver, horizon = Hour(24))

Network Model
┌───────────────┬───────────────────────┐
│ Network Model │ CopperPlatePowerModel │
│ Slacks        │ true                  │
│ PTDF          │ false                 │
│ Duals         │ None                  │
└───────────────┴───────────────────────┘

Device Models
┌────────────────────────┬────────────────────────────┬────────┐
│ Device Type            │ Formulation                │ Slacks │
├────────────────────────┼────────────────────────────┼────────┤
│ RenewableNonDispatch   │ FixedOutput                │ false  │
│ ThermalStandard        │ ThermalBasicUnitCommitment │ false  │
│ PowerLoad              │ StaticPowerLoad            │ false  │
│ InterruptiblePowerLoad │ PowerLoadInterruption      │ false  │
│ RenewableDispatch      │ RenewableFullDispatch      │ false  │
└────────────────────────┴────────────────────────────┴────────┘

Branch Models
┌─────────────────────┬─────────────────────────┬────────┐
│ Branch Type         │ Formulation             │ Slacks │
├─────────────────────┼─────────────────────────┼────────┤
│ Line                │ StaticBranch            │ false  │
│ TapTransformer      │ StaticBranch            │ false  │
│ Transformer2W       │ StaticBranch            │ false  │
│ TwoTerminalHVDCLine │ HVDCTwoTerminalDispatch │ false  │
└─────────────────────┴─────────────────────────┴────────┘

Service Models
┌────────┬──────────────────────────────┬──────────────┬────────┬───────────────
│ Name   │ Service Type                 │ Formulation  │ Slacks │ Aggregated M ⋯
├────────┼──────────────────────────────┼──────────────┼────────┼───────────────
│ REG_UP │ VariableReserve{ReserveUp}   │ RangeReserve │ false  │ false        ⋯
│ REG_DN │ VariableReserve{ReserveDown} │ RangeReserve │ false  │ false        ⋯
│ SPIN   │ VariableReserve{ReserveUp}   │ RangeReserve │ false  │ false        ⋯
└────────┴──────────────────────────────┴──────────────┴────────┴───────────────
                                                                1 column omitted

In [7]:
build!(problem; output_dir = mktempdir())

InfrastructureSystems.Optimization.ModelBuildStatusModule.ModelBuildStatus.BUILT = 0

In [8]:
solve!(problem)

In [ ]:
res = OptimizationProblemResults(problem)

In [ ]:
list_parameter_names(res)
read_parameter(res, "ActivePowerTimeSeriesParameter__RenewableDispatch")